Perguntas

*   Os atributos "imutáveis" (sexo, raça, país de origem...) do trabalhador são menos ou mais determinantes que os demais (como escolaridade) para predizer seu salário?  E para predizer se ele ganha acima de 50 mil dólares anuais?
*   De modo semelhante, qual a menor combinação de atributos que melhor prediz o salário de um trabalhador? E que prediz se ele ganha mais de 50 mil dólares anuais?

*   Com qual precisão conseguimos prever o salário de um trabalhador, baseado em seus atributos, a partir do dataset selecionado? E baseado apenas nos grupos de atributos das perguntas anteriores?

*   Existe algum atributo que é fortemente correlacionado com outros?


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns
import math
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

plt.style.use('seaborn-colorblind')

plt.rcParams['figure.figsize']  = (12, 18)
plt.rcParams['axes.labelsize']  = 20
plt.rcParams['axes.titlesize']  = 20
plt.rcParams['legend.fontsize'] = 20
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['lines.linewidth'] = 4
plt.ion()

In [ ]:
# Load datasets
dataset = pd.read_csv('adult_data.csv')

FileNotFoundError: ignored

In [ ]:
# Arrumando o nome das colunas

dataset.columns

column_new_names = {}
for c in dataset.columns[1:]:
    column_new_names[c] = c.split(' ')[1].replace('-', '_')
    
dataset = dataset.rename(columns=column_new_names)
dataset.head(10)

In [ ]:

# Quantidade de pessoas com salário alto e baixo
pd.DataFrame({'count': dataset.salary.value_counts(), '%': dataset.salary.value_counts(normalize = True)})


# Education

## Distribuição

In [ ]:
#Plot da Escolaridade
column_name = 'education'

new_dataset = dataset.groupby([column_name,'sex'])["age"].count().reset_index(name="count")
all_marital_status = new_dataset[column_name].unique()

b_width = 0.43

men = list()
women = list()
for item in new_dataset.itertuples():
    if item[2] == " Male":
        men.append(item[3])
    else:
        women.append(item[3])

br1 = np.arange(len(men))
br2 = [x + b_width for x in br1]
fig = plt.subplots(figsize=(25,20))

bar1 = plt.bar(br1, men, color ='b', width = b_width,
        edgecolor ='grey', label ='Homens')
bar2 = plt.bar(br2, women, color ='pink', width = b_width,
        edgecolor ='grey', label ='Mulheres')
bar_list = [bar1, bar2]

plt.xlabel('Número de Escolaridade', fontweight ='bold', fontsize = 15)
plt.ylabel('Quantidade', fontweight ='bold', fontsize = 15)

plt.xticks([r + b_width/2 for r in range(len(men))], all_marital_status, fontsize=10, ha="center")
for bars in bar_list:
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x()+ 0.07, yval + .05, yval, fontsize=12, color="red", ha="left")
plt.legend()

## Comparação

In [ ]:
# Education-num

column_name = 'education'
secondary = 'education_num'
#Prepara o dataset
new_dataset = dataset.groupby([column_name,secondary , 'salary'])["education_num"].count().reset_index(name="count")
# all_educations = new_dataset[column_name].unique()

# print(new_dataset)

# Guarda relação nome - número
educations = dict(zip(new_dataset['education'], new_dataset['education_num']))
print(educations)

all_educations = list()
for i in sorted(educations, key = educations.get):
    all_educations.append(i)


key_list = all_educations
value_list = np.zeros(len(all_educations))

education_less_dict = dict(zip(key_list, value_list))
education_more_dict = dict(zip(key_list, value_list))
education_all_dict = dict(zip(key_list, value_list))

# display (new_dataset)

for item in new_dataset.itertuples():
    if item[3] == " <=50K":
        education_less_dict[item[1]] = item[4]
    else:
        education_more_dict[item[1]] = item[4]
    education_all_dict[item[1]] = education_all_dict[item[1]] + item[4]

#print(education_less_dict)
#print(education_more_dict)

# Calcular fração
fraction_less = dict(zip(key_list, value_list))
fraction_more = dict(zip(key_list, value_list))
for key in education_all_dict:
    fraction_less[key] = (education_less_dict[key] / education_all_dict[key])*100
    fraction_more[key] = (education_more_dict[key] / education_all_dict[key])*100

#print(fraction_less)
#print(fraction_more)

#Configura a quantidade de barras e o posicionamento das mesmas.

b_width = 0.35
br1 = np.arange(len(fraction_less.values()))
br2 = [x + b_width for x in br1]

fig = plt.subplots(figsize=(25,10))

#Cria as barras.
bar1 = plt.bar(br1, fraction_less.values(), color ='orange', width = b_width,
        edgecolor ='grey', label ='Porcentagem com <=50k')
bar2 = plt.bar(br2, fraction_more.values(), color ='blue', width = b_width,
        edgecolor ='grey', label ='Porcentagem com >50k')
bar_list = [bar1, bar2]

#Nomeia os eixos.
plt.xlabel('Education', fontweight ='bold', fontsize = 15) #Lembrar de trocar o xlabel se for usar <<<
plt.ylabel('Percentage', fontweight ='bold', fontsize = 15)

#Coloca todas as categorias no eixo X
plt.xticks([r + b_width/2 for r in range(len(fraction_less.values()))], all_educations, fontsize=10, ha="center")
plt.yticks(np.arange(0, 102, 2),fontsize=12)
plt.legend()

In [ ]:
dataset.head(3)

In [ ]:
# Quantidade de pessoas com salário alto e baixo
pd.DataFrame({'count': dataset.salary.value_counts(), '%': dataset.salary.value_counts(normalize = True)})

##Qual é a probabilidade (chance) de que ao acaso 73% dos individos com doutorado ganhem mais de 50K.

valores observado: 

                Doctorate:  25.907990314769975 <=50k
                            74.09200968523002 >50k


In [ ]:
plt.rcParams['figure.figsize']  = (16, 10)
plt.ion()

def despine(ax=None):
    if ax is None:
        ax = plt.gca()
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    # Only show ticks on the left and bottom spines
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')

def sample_proportion100(pop_size, prop, n=10000):
    '''
    Amostra proporções de uma população.
    
    Parâmetros
    ----------
    pop_size: int, tamanho da população
    prop: double, entre 0 e 1
    n: int, número de amostras
    '''
    assert(prop >= 0)
    assert(prop <= 1)
    
    grupo = pop_size * prop
    # print(grupo)
    resultados = np.zeros(n)
    for i in range(n):
        sample = np.random.randint(0, pop_size, 100)
        resultados[i] = np.sum(sample < grupo)
    return resultados


In [ ]:
proporcoes = sample_proportion100(pop_size=32561, prop=0.24081)
print(proporcoes.shape)
bins = np.linspace(1, 100, 100) + 0.5
plt.hist(proporcoes, bins=bins, edgecolor='k')
plt.xlim(0, 80)
plt.ylabel('Numero de Amostras')
plt.xlabel('Porcentagem no grupo')
despine()
for item in fraction_more.values():
    plt.plot([item], [8], 'go', ms=10)
plt.plot([74.09], [8], 'ro', ms=10)
# print(proporcoes)
sample = np.random.randint(0, 32561, 10) # pega 10 inteiros entre 0 e pop_size
print(sample)
print(np.sum(sample < 32561*0.24081)) # ve quantos deles são menores que o esperado

In [ ]:
np.percentile(proporcoes, 95)

Com 95% de chance, no máximo 31% do grupo teria renda maior que 50K. Então, no caso de pessoas com doutorado, que o número foi 74.09, podemos concluir que ele raramente ocorrreria em uma seleção uniforme.

In [ ]:
# print(education_less_dict)
# print(education_all_dict)
# print(education_more_dict)
 
def sample_proportion1000(pop_size, prop, n=10000):
    '''
    Amostra proporções de uma população.
    
    Parâmetros
    ----------
    pop_size: int, tamanho da população
    prop: double, entre 0 e 1
    n: int, número de amostras
    '''
    assert(prop >= 0)
    assert(prop <= 1)
    
    grupo = pop_size * prop
    print("Grupo: ",grupo)
    resultados = np.zeros(n)
    for i in range(n):
        sample = np.random.randint(0, pop_size, 1000) # pega quantos em 1000 já que mem 100 não capturava bem
        resultados[i] = np.sum(sample < grupo)
    return resultados

prop_doct_all = education_more_dict[' Doctorate']/32561
prop_doct_more = education_all_dict[' Doctorate']/7841	
print("Número com doc na pop com >50k:", education_more_dict[' Doctorate']," Número com doc na pop total:" ,education_all_dict[' Doctorate'])
print(prop_doct_all,prop_doct_more)

proporcoes = sample_proportion1000(pop_size=10000000, prop=prop_doct_all)
# print(proporcoes.shape)
bins = np.linspace(1, 100, 100) + 0.5
plt.hist(proporcoes, bins=bins, edgecolor='k')
plt.xlim(0, 55)
plt.ylabel('Numero de Amostras')
plt.xlabel('%% no grupo')
despine()
plt.plot(prop_doct_more*1000, [10], 'ro', ms=10)
print(prop_doct_more*1000)

Com 95% de chance, apenas 0,015% no grupo que ganha mais  de 50K teria doutorado. Então, como o número foi 0,0167%, podemos concluir que isso raramente ocorrreria em uma distribuição uniforme.

In [ ]:
np.percentile(proporcoes, 95)

Em um outro caso em que temos diferentes níveis de educação, podemos computar a total variation distance e fazer outra pergunta.

##Será que a distribuição da educação no grupo que ganha mais de 50k pode ser explicada ao acaso?

In [ ]:
def total_variation(p, q):
    '''
    Computa a total variation distance com base em dois vetore, p e q
    
    Parâmetros
    ----------
    p: vetor de probabilidades de tamanho n
    q: vetor de probabilidades de tamanho n
    '''
    return np.sum(np.abs(p - q)) / 2

In [ ]:
prop_doct_all = education_more_dict[' Doctorate']/32561
prop_doct_more = education_all_dict[' Doctorate']/7841	
# print("Número com doc na pop com >50k:", education_more_dict[' Doctorate']," Número com doc na pop total:" ,education_all_dict[' Doctorate'])
# print(prop_doct_all,prop_doct_more)

idx = list()
for i in education_all_dict.keys():
    idx.append(i)
print(idx)
df = pd.DataFrame(index=idx)


prop_pop_all = list()
for i in education_all_dict.values():
    prop_pop_all.append(i/32561)

df['sample'] = prop_pop_all

# Na amostra de mais de 50K
prop_pop_more = list()
for i in education_more_dict.values():
    prop_pop_more.append(i/7841)

df['pop'] = prop_pop_more


N = 1453
uma_amostra = []
for g in df.index:
    p = df.loc[g]['pop']
    s = sample_proportion100(N, p, 1)[0]
    uma_amostra.append(s/100)


df['1random'] = uma_amostra
df.plot.bar()
plt.ylabel('Propopção')
plt.ylabel('Grupo')
despine()

In [ ]:
total_variation(df['1random'], df['pop'])

In [ ]:
total_variation(df['sample'], df['pop'])

In [ ]:
N = 30000
A = np.zeros(shape=(10000, len(df.index)))
for i, g in enumerate(df.index):
    p = df.loc[g]['pop']
    A[:, i] = sample_proportion100(N, p) / 100

In [ ]:
all_distances = []
for i in range(A.shape[0]):
    all_distances.append(total_variation(df['pop'], A[i])) # total_variation entra a população e 

In [ ]:
plt.hist(all_distances, bins=30, edgecolor='k')
plt.ylabel('Numero de Amostras de Tamanho 10k')
plt.xlabel('Total Variation Distance')
plt.plot([0.255407051893586], [0], 'ro', ms=15)
despine()

Como histograma da TVD vemos no ponto vermelho mostra o valor no grupo que ganha mais de 50k, as barras mostram as diferentes amostras. Novamente como o valor é bastante. Rejeitamos a hipótese nula e indicamos que os dados não foram selecionados de forma uniforme.

In [ ]:
np.percentile(all_distances, 97.5)

Testes de permutação -- Queremos demonstrar se o fato de um gênero ganhar mais que outro pode ou não ser justificado pelo acaso.

Assuma uma significância de 5%


In [ ]:
def permutation_test(df, sig, rep_num, col, attribute):
  """
  Parâmetros:
  ------------
  df: Dataframe utilizado para realizar o teste de permutação.
  sig: Valor da significância.
  rep_num: Quantidade de repetições do experimento.
  col: Coluna alvo do DF para ser realizada o teste de permutação.
  attribute: Valor da coluna alvo para a permutação ser realizada.
 
  Retornos:
  ------------
  diffs: Vetor que contém a diferença dos valores analisados.
  LI: Ponto crítico inferior.
  LS: Ponto crítico superior.
 
  """
  
  N = rep_num
  filtro = df[col] == attribute
  diffs = np.zeros(N)
  sig = sig/2
  for i in range(N):
    np.random.shuffle(filtro.values)
    #df['filtro'] = np.random.permutation(df.filtro)
    #men = df.loc[((df['salary'] == " <=50K") & (df['filtro'] == 1))].shape[0]
    #women = df.loc[((df['salary'] == " <=50K") & (df['filtro'] == 0))].shape[0]
    diffs[i] = df.loc[df['salary'] == ' <=50K'][filtro].shape[0] - df.loc[df['salary'] == ' <=50K'][~filtro].shape[0]
    #diffs[i] = df[df.salary == ' <=50K'][df.filtro == 1].count().iloc[0] - \
    #df[df.salary == ' <=50K']['df.filtro == 0'].count().iloc[0]
  LI = np.percentile(diffs, sig)
  LS = np.percentile(diffs, 100-sig)
  
  return diffs, LI, LS

In [ ]:
sigma = 5
rep_num = 5000
column= "sex"
at = " Male"
difference_vector, LI,LS = permutation_test(dataset, sigma, rep_num,column,at)


In [ ]:
ab = dataset.query('salary == " <=50K"')
male = ab[ab.sex == ' Male'].count().iloc[0]
female = ab[ab.sex == ' Female'].count().iloc[0]
sample_difference = male-female
 
 
plt.hist(difference_vector, bins=30, edgecolor='k')
plt.ylabel('Quantidade')
plt.xlabel('Diferença entre o número de Homens e Mulheres que ganham abaixo de 50K Dol')
plt.plot(sample_difference, [0], 'ro', ms=15)
#Usando np.permutation


In [ ]:
ab = dataset.query('salary == " <=50K"')
male = ab[ab.sex == ' Male'].count().iloc[0]
female = ab[ab.sex == ' Female'].count().iloc[0]
sample_difference = male-female
 
 
plt.hist(difference_vector, bins=30, edgecolor='k')
plt.ylabel('Quantidade')
plt.xlabel('Diferença entre o número de Homens e Mulheres que ganham abaixo de 50K Dol')
plt.plot(sample_difference, [0], 'ro', ms=15)
#Usando np.shuffle

In [ ]:
sigma = 5
rep_num = 5000
column= "marital_status"

target_values = [' Divorced', ' Married-civ-spouse']
df = dataset.query("marital_status == ' Never-married' or marital_status == ' Married-civ-spouse'")


#new_dataset = dataset.loc[(dataset['marital_status'].isin(target_values))]
at = " Married-civ-spouse"
diff1, li1,ls1 = permutation_test(df, sigma, rep_num,column,at)



In [ ]:
ab = dataset.query('salary == " <=50K"')
nm = ab[ab.marital_status == ' Married-civ-spouse'].count().iloc[0]
mcs = ab[ab.marital_status == ' Never-Married'].count().iloc[0]
sample_difference = nm-mcs
 
plt.hist(diff1, bins=10, edgecolor='k')
plt.ylabel('Quantidade')
plt.xlabel('Diferença entre o número de Casados e Não Casados que ganham abaixo de 50K Dol')
plt.plot(sample_difference, [0], 'ro', ms=15)


# Regressões

##Regressão Linear

##Regressão Múltipla

In [ ]:
df = dataset.copy()
df = df.replace('?', np.nan)
df = df.dropna()

df = df.drop(['fnlwgt','workclass','education','marital_status','occupation','relationship','race','sex','native_country','salary'], axis=1)
X = df.drop('age', axis=1)
y = df[['age']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

reg = LinearRegression()
reg.fit(X_train[['hours_per_week','education_num','capital_gain','capital_loss']], y_train)
y_predicted = reg.predict(X_test[['hours_per_week','education_num','capital_gain','capital_loss']])
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_predicted))
print('R²: %.2f' % r2_score(y_test, y_predicted))

fig, ax = plt.subplots()
ax.scatter(y_test, y_predicted)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
ax.set_xlabel('measured')
ax.set_ylabel('predicted')
plt.show()

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

data = dataset.drop(columns=['fnlwgt', 'education'])
data = data.replace('?', np.nan)
data = data.dropna()
data['capital'] = data['capital_gain'] - data['capital_loss']
data = data.drop(columns=['capital_gain', 'capital_loss'])

categoricalColumns = data.columns[data.dtypes==object].tolist()

for col in categoricalColumns:
    data[col] = data[col].str.strip()

for col in categoricalColumns:
    data[col] = data[col].str.replace('-', '_')

data.loc[data['native_country'] != 'United_States', 'native_country'] = 'Other'
data.loc[data['race'] != 'White', 'race'] = 'Other'
data['salary'] = data['salary'].str.replace('<=50K', 'low')
data['salary'] = data['salary'].str.replace('>50K', 'high')

print(data.dtypes)
print(data)

data.head()

formula_string_indep_vars = ' + '.join(dataset.drop(columns='hours_per_week').columns)
formula_string = 'hours_per_week ~ ' + formula_string_indep_vars
print('formula_string: ', formula_string)

data_encoded = pd.get_dummies(data, drop_first=True)

formula_string_indep_vars_encoded = ' + '.join(data_encoded.drop(columns='hours_per_week').columns)
formula_string_encoded = 'hours_per_week ~ ' + formula_string_indep_vars_encoded
print('formula_string_encoded: ', formula_string_encoded)

#formula_string_encoded = formula_string_encoded + ' + hours_per_week:capital + race_White:capital'

#formula_string_encoded = formula_string_encoded + ' + np.power(hours_per_week, 1)'

model_full = sm.formula.ols(formula=formula_string_encoded, data=data_encoded)

model_full_fitted = model_full.fit()

print(model_full_fitted.summary())

residuals_full = pd.DataFrame({'actual': data_encoded['hours_per_week'], 
                            'predicted': model_full_fitted.fittedvalues, 
                            'residual': model_full_fitted.resid})

def plot_line(axis, slope, intercept, **kargs):
    xmin, xmax = axis.get_xlim()
    plt.plot([xmin, xmax], [xmin*slope+intercept, xmax*slope+intercept], **kargs)
    
# Creating scatter plot
plt.scatter(residuals_full['actual'], residuals_full['predicted'], alpha=0.3);
plot_line(axis=plt.gca(), slope=1, intercept=0, c="red");
plt.xlabel('Actual hours_per_week');
plt.ylabel('Predicted Age');
plt.title('Figure 9: Scatter plot of actual vs. predicted age for the Full Model', fontsize=15);
plt.show();

##Pipeline Machine Learning

In [ ]:
import copy
df = dataset.copy()
df.head()

In [ ]:
##Separando os atributos numéricos dos categóricos, para Z normalizá-los.

numerical_categories = list()
for coluna in df.columns:
  if (df.dtypes[coluna] == 'object'):
    continue
  else:
    numerical_categories.append(coluna)

print(numerical_categories)


In [ ]:
##Separando o conjunto de treino e teste para o ML.

from sklearn.model_selection import train_test_split

## A coluna salário é guardada na variável y, que será o atributo previsto. A variável x guardará o resto os outros parâmetros que serão usados para prever o salário. 
## Lembrando que o dataset já veio com um arquivo de teste separado. Ignorar é prudente, pois não é possivel saber se foi retirado do adult_data, ou se corresponde
## á um conjunto de dados adicionais. Caso seja a primeira opção, vai avacalhar tudo! :(

y = df['salary']
x = df.copy()
x = x.drop('salary', axis=1)




x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

In [ ]:
##x_train_Znormalizado

# mean_train = x_train[numerical_categories].mean()
# std_train = x_train[numerical_categories].std(ddof=1)

x_trn = x_train.copy()
x_trn[numerical_categories] -= x_train[numerical_categories].mean()
x_trn[numerical_categories] /= x_train[numerical_categories].std(ddof=1)
x_trn
##x_test_Znormalizado.
x_ten = x_test.copy()
x_ten[numerical_categories] -= x_test[numerical_categories].mean()
x_ten[numerical_categories] /= x_test[numerical_categories].std(ddof=1)
x_ten

In [ ]:
x_ten

In [ ]:
##Fazendo one-hot-encoding.
x_trn = pd.get_dummies(x_trn)
x_ten = pd.get_dummies(x_ten)
x_trn.drop(columns=x_trn.columns[-1], inplace=True)



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
#Criar um conjunto de validação.. StratifiedKFold mantém a proporção original de dados do dataset nos folds criados.
ns = 3
skf = StratifiedKFold(n_splits=ns)
tururu = 0
for train_i, validation_i in skf.split(x_trn, y_train):
  for k in [1, 3, 5, 7, 9, 11, 13, 15, 17]:
    #Treinar o KNN
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(x_trn.iloc[train_i], y_train.iloc[train_i])
    #Usar o KNN para prever, utilizando a parte de validação do conjunto de treino.
    predict = model.predict(x_trn.values[validation_i])
    original = y_train.values[validation_i]
    #Calcular a eficiência do modelo preliminar durante os ajustes dos parâmetros 'k'.
    for index in range(len(predict)):
      if predict[index] == original[index]:
        tururu +=1 
    tururu /= len(predict)
    print(k, tururu)
##13 parece bom.

In [ ]:
ns = 10
k = 13
skf = StratifiedKFold(n_splits=ns)
for train_i, validation_i in skf.split(x_trn, y_train):
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(x_trn.iloc[train_i], y_train.iloc[train_i])
    xd2 = model.score(x_trn.values[validation_i], y_train.values[validation_i])
    lr = LogisticRegression(max_iter=10000)
    lr.fit(x_trn.iloc[train_i], y_train.iloc[train_i])
    xd1 =lr.score(x_trn.values[validation_i], y_train.values[validation_i])
    print(f"Log:{xd1}\nKNN:{xd2}")

In [ ]:
##Bootstrap
from sklearn.metrics import accuracy_score
def bootstrap(x, y, model, n=5000):
  size = len(x)
  values = np.zeros(n)
  idx = np.arange(len(y))
  for i in range(n):
    sample = np.random.choice(idx, size=size, replace=True)
    values[i] = model.score(x.values[sample], y.values[sample])
  return values

In [ ]:
##LR
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=10000)
lr.fit(x_trn, y_train)

precisao_final = lr.score(x_ten, y_test)
print(precisao_final)


In [ ]:
boots = bootstrap(x_ten, y_test, lr)
print(boots)

In [ ]:
LL = np.percentile(boots, 5)
RL = np.percentile(boots,95)
print(f"O intervalo de confiança da regressão logística é: [{LL}, {RL}]")
